In [2]:
from timeit import default_timer as timer

In [70]:
#!wget -N https://files.rcsb.org/download/1TGH.pdb -O 1TGH.pdb
#!wget -N https://files.rcsb.org/download/1CDW.pdb
!wget -c https://raw.githubusercontent.com/moqri/DNA_molecular_dynamics/master/1tgh-5.pdb -O 1TGH.pdb

--2020-04-22 15:33:34--  https://raw.githubusercontent.com/moqri/DNA_molecular_dynamics/master/1tgh-5.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [71]:
#interactive 
import pytraj as pt
import nglview as nv

traj = pt.load('1TGH.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
view

NGLWidget()

In [72]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
pdb = PDBFile('1TGH.pdb')
pdb.getTopology()

<Topology; 3 chains, 204 residues, 1915 atoms, 1999 bonds>

In [90]:
from pdbfixer import PDBFixer
fixer = PDBFixer(filename='1TGH.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
#fixer.addSolvent(fixer.topology.getUnitCellDimensions())
fixer.addSolvent(padding=10*angstrom)
PDBFile.writeFile(fixer.topology, fixer.positions, open('1TGH_fixed.pdb', 'w'))

In [91]:
fixer.topology

<Topology; 5 chains, 14805 residues, 47507 atoms, 32990 bonds>

In [92]:
traj = pt.load('1TGH_fixed.pdb')
pt.rotate(traj, 'y 90')
view 

NGLWidget(n_components=1, picked={'atom1': {'index': 1797, 'residueIndex': 198, 'resname': 'DT', 'x': 16.72599…

In [50]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#system = forcefield.createSystem(fixer.topology)
system = forcefield.createSystem(fixer.topology, nonbondedCutoff=1*nanometer, constraints=HBonds, 
                                 nonbondedMethod=app.NoCutoff)#,nonbondedMethod=PME,
simulation = Simulation(fixer.topology, system, integrator)
simulation.context.setPositions(fixer.positions)
start = timer()
simulation.minimizeEnergy()
timer()-start

147.36914913200008

In [51]:
simulation.reporters.append(PDBReporter('1TGH_fixed_md.pdb',100))
simulation.reporters.append(StateDataReporter(stdout, 100, step=True,potentialEnergy=True, temperature=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-854315.0,54.77718091350822
200,-836224.5,99.20032317538295
300,-821387.25,135.40478006976664
400,-808539.875,164.7608061127961
500,-798157.625,188.42944795204372
600,-789697.0,208.55774958706158
700,-781193.0625,221.77779821983017
800,-775717.25,236.42118275650978
900,-770324.125,245.04993550836943
1000,-765938.6875,253.49526945253115
1100,-762525.5,262.48222841156996
1200,-758944.25,266.7726531175099
1300,-756074.0625,271.75856373408834
1400,-754176.25,276.94150620660844
1500,-751726.4375,279.1500593084453
1600,-749299.5625,281.6803123184956
1700,-747912.5,284.2319660293006
1800,-746726.625,287.27373002412173
1900,-745859.375,288.4091147883328
2000,-744402.125,289.20082686509886
2100,-743773.5,290.55046285564293
2200,-742594.1875,290.9579324214575
2300,-742110.25,295.0441238460271
2400,-740261.4375,294.2527073126853
2500,-740479.1875,297.22289965913467
2600,-739772.5625,295.1404681612308
2700,-738989.125,294.841904139978
2800

In [93]:
traj = pt.load('1TGH_fixed_md.pdb')
pt.rotate(traj, 'y 90')
view = nv.show_pytraj(traj)
#view.add_ball_and_stick('not protein')
view

NGLWidget(max_frame=99)